In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import plotly.express as px
from sklearn import *
import lightgbm as lgbm 
import catboost as ctb
import re
from tqdm import tqdm
import warnings
from collections import *
warnings.filterwarnings('ignore')
tqdm.pandas()
plt.style.use('ggplot')

In [4]:
df = pd.read_csv(r'clean_df.csv')
df = df.dropna()

In [5]:
df

,ttext,ttype,preprocessed,clean_text
0,"@first_timee хоть я и школота, но поверь, у на...",1,"['хот', 'я', 'и', 'школот', 'но', 'повер', 'у'...",хот я и школот но повер у нас то же сам общест...
1,"Да, все-таки он немного похож на него. Но мой ...",1,"['да', 'все', 'так', 'он', 'немн', 'похож', 'н...",да все так он немн похож на нег но мо мальчик ...
2,RT @KatiaCheh: Ну ты идиотка) я испугалась за ...,1,"['ну', 'ты', 'идиотк', 'я', 'испуга', 'за', 'т...",ну ты идиотк я испуга за теб
3,"RT @digger2912: ""Кто то в углу сидит и погибае...",1,"['кто', 'то', 'в', 'угл', 'сид', 'и', 'погиба'...",кто то в угл сид и погиба от голод а мы ещ пор...
4,@irina_dyshkant Вот что значит страшилка :D\nН...,1,"['вот', 'что', 'знач', 'страшилк', 'но', 'блин...",вот что знач страшилк но блин посмотр все част...
...,...,...,...,...
226829,Но не каждый хочет что то исправлять:( http://...,-1,"['но', 'не', 'кажд', 'хочет', 'что', 'то', 'ис...",но не кажд хочет что то исправля
226830,скучаю так :-( только @taaannyaaa вправляет мо...,-1,"['скуча', 'так', 'тольк', 'вправля', 'мозг', '...",скуча так тольк вправля мозг но я все равн скуча
226831,"Вот и в школу, в говно это идти уже надо(",-1,"['вот', 'и', 'в', 'школ', 'в', 'говн', 'эт', '...",вот и в школ в говн эт идт уж над
226832,"RT @_Them__: @LisaBeroud Тауриэль, не грусти :...",-1,"['тауриэл', 'не', 'груст', 'обня']",тауриэл не груст обня


In [23]:
def nlp_ex(text):
    # some feature engineering
    upper = [[hatred_upper(text)]]
    punctuation = [[hatred_punc(text)]]
    title = [[len([i for i in text if i.istitle()==True])]]
    pos_sc = [[len(re.findall(r'\)|D',text))]]
    neg_sc = [[len(re.findall(r'\(|C|c|С|c',text))]]
    #text preprocessing
    stemmer = nltk.stem.SnowballStemmer('russian')
    preproc=[stemmer.stem(i) for i in nltk.regexp_tokenize(text.lower(),r'[А-я]+')]
    text = ' '.join(preproc)
    idf_text= (idf.transform([text])).toarray()
    # concat type of file that we needeed
    end_text=np.hstack([idf_text,upper,punctuation,title,pos_sc,neg_sc])
    # use model
    pred = clf_mnb.predict_proba(end_text)

    return print(f" Negative {pred[0][0]:.2%}, Positive {pred[0][1]:.2%}")

In [24]:
for i in ['эт хорош','позитив',"понравил вечер"," эт прост моя любов","милота"]:
    nlp_ex(i)

Negative 37.05%, Positive 62.95%
 Negative 49.34%, Positive 50.66%
 Negative 30.67%, Positive 69.33%
 Negative 47.14%, Positive 52.86%
 Negative 49.34%, Positive 50.66%


In [26]:
for i in ['ужас','ну что за день(',"ненавижу тебя","прост тварь","это просто фиаско!"]:
    nlp_ex(i)

Negative 82.39%, Positive 17.61%
 Negative 92.15%, Positive 7.85%
 Negative 80.94%, Positive 19.06%
 Negative 50.57%, Positive 49.43%
 Negative 48.13%, Positive 51.87%
